In [1]:
from Assemblies.game import GameField
import numpy as np

from fractions import Fraction

%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.animation as animation

np.set_printoptions(precision=3)

In [2]:
def string_field(field):
    """Poor-man's viz of the game field."""
    s_rows = ["".join(str(row)[1:-1].split()) for row in field]
    return "\n".join(s_rows)

In [3]:
"""Candidate fractions (num & den <= 7)."""
all_fracs = set()
for den in range(1, 7+1):
    for num in range(1, 7+1):
        all_fracs.add(Fraction(num, den))
fracs = [f for f in all_fracs if 1 < f < 3]
fracs = sorted(list(fracs))
fracs

[Fraction(7, 6),
 Fraction(6, 5),
 Fraction(5, 4),
 Fraction(4, 3),
 Fraction(7, 5),
 Fraction(3, 2),
 Fraction(5, 3),
 Fraction(7, 4),
 Fraction(2, 1),
 Fraction(7, 3),
 Fraction(5, 2)]

In [4]:
"""Candidate fractions (num & den <= 9)."""
all_fracsQ = set()
for den in range(1, 9+1):
    for num in range(1, 9+1):
        all_fracsQ.add(Fraction(num, den))
fracsQ = [f for f in all_fracsQ if 1 < f < 3]
fracsQ = sorted(list(fracsQ))
fracsQ

[Fraction(9, 8),
 Fraction(8, 7),
 Fraction(7, 6),
 Fraction(6, 5),
 Fraction(5, 4),
 Fraction(9, 7),
 Fraction(4, 3),
 Fraction(7, 5),
 Fraction(3, 2),
 Fraction(8, 5),
 Fraction(5, 3),
 Fraction(7, 4),
 Fraction(9, 5),
 Fraction(2, 1),
 Fraction(9, 4),
 Fraction(7, 3),
 Fraction(5, 2),
 Fraction(8, 3)]

In [5]:
# init
L = 30
fraction_D = 0.1

In [6]:
def switch_time(b, num_steps=1000, num_therm=1000, seed=12388):

    rndm = np.random.RandomState(seed)

    # Init the field
    instance = (rndm.uniform(size=(L, L)) < fraction_D).astype(int)
    game = GameField(L, b)
    game.field = instance

    # burn-in
    evolve = game.evolve
    evolve(num_therm)

    snapshots = np.zeros((L, L, num_steps), dtype=int)
    snapshots[:, :, 0] = game.field

    for step in range(num_steps):
        evolve(1)
        snapshots[:, :, step] = game.field

    # compute the switch times
    switches = np.zeros_like(snapshots[:, :, 0], dtype=int)

    for i in range(snapshots.shape[0]):
        for j in range(snapshots.shape[1]):
            s = snapshots[i, j, :]
            idx, = (s[1:] - s[:-1]).nonzero()
            switches[i, j] = idx.size + 1

    return np.mean(num_steps / switches), switches

In [7]:
def compute_times(fracs, num_steps=1000, num_therm=1000, seed=12388):
    mean_tau = dict()
    for frac in fracs:
        b = frac + 1e-3
        mean_tau[frac], _ = switch_time(b, num_steps, num_therm, seed)
        print(frac, mean_tau[frac])
    return mean_tau

In [8]:
def compute_times_and_volumes(fracs, num_steps=1000, num_therm=1000, seed=12388):
    mean_tau, num_swingers, tau_swingers = dict(), dict(), dict()
    for frac in fracs:
        b = frac + 1e-3
        mean_tau[frac], switches = switch_time(b, num_steps, num_therm, seed)
        
        swingers = switches > 1
        num_swingers[frac] = np.count_nonzero(swingers)
        tau_swingers[frac] = np.mean(num_steps / switches[swingers])
        print(frac, mean_tau[frac], ' - ', num_swingers[frac], ' - ', tau_swingers[frac])
    return mean_tau, num_swingers, tau_swingers

#print("============ T: ")
#tauT, num_swingersT, tau_swingersT = compute_times_and_volumes(fracs, num_steps=int(2e4))
#print("============ Q: ")
#tauQ, num_swingersQ, tau_swingersQ = compute_times_and_volumes(fracsQ, game_type="Q", num_steps=int(2e4))

In [9]:
num_seeds = 3

tauT_all, num_swT_all, tau_swT_all = [], [], []
tauQ_all, num_swQ_all, tau_swQ_all = [], [], []

for seed in [11]: #, 12388, 123]:
    print("========== seed = ", seed)
    tauT, num_swingersT, tau_swingersT = compute_times_and_volumes(fracs, num_steps=int(10000))
    tauT_all.append(tauT)
    num_swT_all.append(num_swingersT)
    tau_swT_all.append(tau_swingersT)

========== seed =  11
7/6 8200.18  -  162  -  1.0
6/5 9144.53  -  77  -  1.0
5/4 8977.88  -  92  -  1.0
4/3 9866.68  -  12  -  1.0
7/5 8300.17444377781  -  153  -  1.0261398694771144
3/2 9288.9666653336  -  64  -  1.09373125374925
5/3 9366.771658085661  -  57  -  1.6577592472881677
7/4 6361.302207970181  -  348  -  589.5746757849531
2 9611.16222038898  -  35  -  1.3142385737855966


C:\Users\Admin\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Admin\Anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


7/3 10000.0  -  0  -  nan
5/2 10000.0  -  0  -  nan


In [10]:
num_seeds = 3

tauT_all, num_swT_all, tau_swT_all = [], [], []
tauQ_all, num_swQ_all, tau_swQ_all = [], [], []

for seed in [11]: #, 12388, 123]:
    print("========== seed = ", seed)
    tauT, num_swingersT, tau_swingersT = compute_times_and_volumes(fracs, num_steps=int(10000),num_therm=int(5000))
    tauT_all.append(tauT)
    num_swT_all.append(num_swingersT)
    tau_swT_all.append(tau_swingersT)

========== seed =  11
7/6 8200.18  -  162  -  1.0
6/5 9144.53  -  77  -  1.0
5/4 8977.88  -  92  -  1.0
4/3 9866.68  -  12  -  1.0
7/5 8300.17444377781  -  153  -  1.0261398694771144
3/2 9288.9666653336  -  64  -  1.09373125374925
5/3 9366.77165708636  -  57  -  1.6577434688609607
7/4 9911.12  -  8  -  1.0
2 9611.16222038898  -  35  -  1.3142385737855966


C:\Users\Admin\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Admin\Anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


7/3 10000.0  -  0  -  nan
5/2 10000.0  -  0  -  nan
